# 기초 통계 및 경영 통계 시험(10.20)

In [7]:
import pandas as pd

1. 데이터 로딩

In [8]:
# 1번. 데이터 로딩
df = pd.read_csv("./문제해결_양지후/cdn_page_load_times.csv")
df.head()

,CDN_A,CDN_B,CDN_C
0,103.115543,102.342433,98.371838
1,94.341060,118.835131,86.193353
2,114.586739,104.359728,79.047870
3,156.155017,90.357875,63.730201
4,156.920524,75.013771,116.655484


2. 로딩한 데이터를 분석하기에 적합한 가설 검정 방법의 선택 : 일원분산 anova 분석 사용
> 이유 : 셋 이상의 그룹 간의 차이를 확인하는 것이 필요하기 때문

3. 정규성 검정 및 등분산성(레빈 검정)이 필요합니다. (아래에 수행)

In [9]:
# 데이터 분할 (결측이 포함되어 있을 수 있으므로 dropna를 사용)
A = df['CDN_A'].dropna().values
B = df['CDN_B'].dropna().values
C = df['CDN_C'].dropna().values

In [12]:
#정규성 검정을 위한 Shapiro-Wilk 검정
from scipy import stats
print(stats.shapiro(A-B,))
print(stats.shapiro(A-C))
print(stats.shapiro(B-C))

ShapiroResult(statistic=np.float64(0.9940977465531138), pvalue=np.float64(0.9446069376138456))
ShapiroResult(statistic=np.float64(0.9898981527862001), pvalue=np.float64(0.6563609451801027))
ShapiroResult(statistic=np.float64(0.9821121731681786), pvalue=np.float64(0.19351085781526467))


[결과해석]
- 모두 p-value값이 0.05이상이므로, 정규분포를 따른다 > 귀무가설 "데이터가 정규 분포를 따른다"선택
- 순서대로 p-value : 0.9446, 0.6563, 0.935

In [13]:
#등분산성 검정을 위한 레빈 검정
levene_stat, levene_p_value = stats.levene(A, B, C, center='median')
print(f"levene_stat: {levene_stat}, levene_p_value: {levene_p_value}")

levene_stat: 0.6331383822746123, levene_p_value: 0.5316379878247686


[결과해석]
-  p-value값이 0.05이상이므로, 귀무가설 "모든 그룹의 분산이 같다" 만족
- p-value : 0.5316

4. 가설검정 수행 및 결과 해석

In [14]:
#일원분산분석 수행 및 결과 해석
print(stats.f_oneway(A, B, C))

F_onewayResult(statistic=np.float64(39.192607237897704), pvalue=np.float64(7.842852500131944e-16))


[결과 해석]
결론 : 그룹 간 평균에 유의한 차이가 있다!

- p-value = 7.842/10에 16승 < 0.05 : 매우 작은 p-value 값을 보이므로, 강력한 유의성이 보이고, 이에 귀무 가설을 기각

즉, 어느 CDN서비스인지는 몰라도, 전체 CDN서비스의 속도 평균이 동일하지 않다. 적어도 하나는 다르다!

5. 사후검정
- 앞서 다르다는 것을 확인했기 때문에, 어떤 CDN서비스가 다른지 확인하는 사후 검정이 필요하다.

In [15]:
#사후분석
from statsmodels.stats import multicomp 

Data = A.tolist() + B.tolist() + C.tolist()
Group = ['A'] * len(A) + ['B'] * len(B) + ['C'] * len(C)

print(multicomp.pairwise_tukeyhsd(Data, Group))

 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower    upper  reject
-----------------------------------------------------
     A      B -24.8154    0.0 -31.4178 -18.213   True
     A      C -12.5563    0.0 -19.1587 -5.9538   True
     B      C  12.2591 0.0001   5.6567 18.8616   True
-----------------------------------------------------


[결과 해석]
* 세 그룹이 전부다 유의한 차이가 있다(reject = "True")
* 결과 값에 드러난 '차이'에 따라 순위를 세우면, A > c > B의 순서이다.
> B가 A보다 24.8 낮음, C가 A보다 12.6 낮음, C가 B보다 12..3 높음